<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    SAN Projekt:<br> Schweizer Offshore Firmen
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Florin Barbisch, Gabriel Torres Gamez und Tobias Buess im FS 2024.
</p>

Wir führen eine Voranalyse für das Bundesamt für Statistik durch, um die kürzlich aufgetretenen Leaks aus den Offshore Papers zu untersuchen. 

Diese Analyse zielt darauf ab, Umfang und Natur der Verbindungen in Schweizer Offshore-Strukturen zu ermitteln. Wir verwenden dafür Daten aus der [Offshore Leaks Database](https://offshoreleaks.icij.org/), um mögliche Muster, wichtige Personen aufzudecken, die für die Steuerbehörden oder Regulierungsorgane von Interesse sein könnten. 

Unsere Arbeit umfasst eine detaillierte Prüfung der betroffenen Entitäten. Dies wird es dem Bundesamt für Statistik ermöglichen, fundierte Entscheidungen zur weiteren Untersuchung und möglichen Massnahmen zu treffen.

## Inhalt
1. Wie sehen die Daten aus? Was für Informationen können wir daraus ziehen?
2. Grobe Metriken zu Schweizer Officers und deren Offshore Firmen.
3. ...

In [1]:
import os
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import re

# Daten laden

In [2]:
def remove_special_characters(text):
    # Stellen Sie sicher, dass der Text ein String ist
    if isinstance(text, str):
        # Entfernt alles außer Buchstaben, Ziffern, Leerzeichen und grundlegenden Satzzeichen
        return re.sub(r'[^\w\s,.]', '', text)
    return text

if not os.path.exists("./data/graph.gexf"):
    nodes_addresses = pd.read_csv("./data/nodes-addresses.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
    nodes_addresses["node_type"] = ["Address"]*len(nodes_addresses)

    nodes_entities = pd.read_csv("./data/nodes-entities.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
    nodes_entities["node_type"] = ["Entity"]*len(nodes_entities)

    nodes_intermediaries = pd.read_csv("./data/nodes-intermediaries.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
    nodes_intermediaries["node_type"] = ["Intermediary"]*len(nodes_intermediaries)

    nodes_officers = pd.read_csv("./data/nodes-officers.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
    nodes_officers["node_type"] = ["Officer"]*len(nodes_officers)

    nodes_others = pd.read_csv("./data/nodes-others.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
    nodes_others["node_type"] = ["Other"]*len(nodes_others)

    relationships = pd.read_csv("./data/relationships.csv", low_memory=False).set_index(["node_id_start", "node_id_end"]).astype(str).map(remove_special_characters)

    G = nx.MultiDiGraph() #directed multi-edge graph
    G.add_nodes_from([(key, value) for key, value in nodes_addresses.to_dict("index").items()])
    G.add_nodes_from([(key, value) for key, value in nodes_entities.to_dict("index").items()])
    G.add_nodes_from([(key, value) for key, value in nodes_intermediaries.to_dict("index").items()])
    G.add_nodes_from([(key, value) for key, value in nodes_officers.to_dict("index").items()])
    G.add_nodes_from([(key, value) for key, value in nodes_others.to_dict("index").items()])
    G.add_edges_from([(*relationships.index[i], value) for i, value in enumerate(relationships.to_dict(orient='records'))])

    del nodes_addresses
    del nodes_entities
    del nodes_intermediaries
    del nodes_officers
    del nodes_others
    del relationships

    nx.write_gexf(G, "./data/graph.gexf")

    del G

graph = nx.read_gexf("./data/graph.gexf")

## Wie sehen die Daten aus? Was für Informationen können wir daraus ziehen?

## Grobe Metriken zu schweizer Officers und deren Offshore Firmen.

## ...